# LSTM model 1 on KaggleCompetition data set 

## Importing packages 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout,concatenate
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model 
from sklearn.model_selection import train_test_split
import sys  
sys.path.insert(0, '../../../models')
from PY.model1 import get_model1

Using TensorFlow backend.


## Paths 

In [2]:
DATA_DIR = '../../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model1_Kaggle.h5'

## Loading Data 

In [3]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Politifact/Claims_to_work_with/claims.csv'))
train

author  label  \
0                                            Paul LePage      1   
1                                     Battleground Texas      0   
2                                     Battleground Texas      0   
3                                         Suzanne Somers      0   
4                                           Jamie Oliver      0   
...                                                  ...    ...   
10459                                       Bill O'Neill      1   
10460                                       Presente.org      0   
10461                                       Presente.org      1   
10462  Local 799 of the International Assoc. of Fire ...      1   
10463                                     Jason Chaffetz      1   

                                                   title  
0      About 47 percent of able-bodied people in the ...  
1      Says Dan Patrick has "called immigration into ...  
2      In 2008, "only 54 percent of Latinos in Texas ...  
3      Even after Obamacare is fully implemented, the...  
4      McDonald's in England only sells organic milk ...  
...                                                  ...  
10459  In recent years, Rep. LaTourette has sponsored...  
10460  91 percent of Latinos support the DREAM Act, w...  
10461  Rep. Adam Putnam was silent when Rush Limbaugh...  
10462  Elorza wants to teach our public school childr...  
10463  We had people that were getting killed (in Ben...  

[10464 rows x 3 columns]

In [4]:
train = train[train['author'].notna()]

In [5]:
train=train.reset_index()

In [6]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [7]:
size_comb=[0.05,0.1, 0.15, 0.2, 0.25, 0.3]
result_list=[]

In [ ]:
#Main program 
for comb in (size_comb):
    test_size_value=comb
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=comb)
    
    # set vocabulary size
    vo_size=500
    messages=X_train.copy()
    messages.reset_index(inplace=True)
    
    # Dataset Preprocessing
    ps =PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        print("Status: %s / %s" %(i, len(messages)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
         
    # one hot representation
    onehot_rep = [one_hot(words, vo_size) for words in corpus]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
    print(embedded_doc)
    embedding_vector_feature = 10
    
    
    messages1=X_test.copy()
    messages1.reset_index(inplace=True)

    
    
    # Dataset Preprocessing
    
    ps1 =PorterStemmer()
    corpus1 = []
    for i in range(0, len(messages1)):
        print("Status: %s / %s" %(i, len(messages1)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages1['title'][i])
        review = review.lower()
        review = review.split()
    
        review = [ps1.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus1.append(review)
        
        
    
    # one hot representation
    onehot_rep1 = [one_hot(words, vo_size) for words in corpus1]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc1=pad_sequences(onehot_rep1, padding='pre', maxlen=sent_length)
    print(embedded_doc1)
    
    
    # final data for NN
    X_final_train=np.array(embedded_doc)
    y_final_train=np.array(y_train)
    X_final_train.shape,y_final_train.shape
    
    X_final_test=np.array(embedded_doc1)
    y_final_test=np.array(y_test)
    X_final_test.shape,y_final_test.shape


    
    model = Sequential()
    model.add(Embedding(vo_size,embedding_vector_feature,input_length=sent_length))
    model.add(LSTM(100))
    model.add(Dense(1,activation='sigmoid')) 
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model 
    history=model.fit(X_final_train,y_final_train,validation_split=0.1,epochs=10,batch_size=64)
    # Evaluate the model 
    results = model.evaluate(X_final_test, y_final_test, batch_size=64)
    # Stock the results 
    item=(comb, y_final_train.shape, y_final_test.shape,results)
    result_list.append(item)

[[  0   0   0 ... 131  73 415]
 [  0   0   0 ...  46 338 191]
 [  0   0   0 ... 472 258  39]
 ...
 [  0   0   0 ... 360 246 189]
 [  0   0   0 ... 352 289  78]
 [  0   0   0 ... 329 325 462]]
[[  0   0   0 ... 439 307 160]
 [  0   0   0 ...  84 335 206]
 [  0   0   0 ...  64 403 258]
 ...
 [  0   0   0 ... 258 170 392]
 [  0   0   0 ... 346 308 406]
 [  0   0   0 ... 179  42  14]]
Train on 8946 samples, validate on 994 samples
Epoch 1/10
8946/8946 [==============================] - 9s 964us/sample - loss: 0.6849 - accuracy: 0.5629 - val_loss: 0.6721 - val_accuracy: 0.5835
Epoch 2/10
8946/8946 [==============================] - 5s 532us/sample - loss: 0.6632 - accuracy: 0.6045 - val_loss: 0.6680 - val_accuracy: 0.5865
Epoch 3/10
8946/8946 [==============================] - 5s 514us/sample - loss: 0.6481 - accuracy: 0.6262 - val_loss: 0.6667 - val_accuracy: 0.5815
Epoch 4/10
4416/8946 [=============>................] - ETA: 2s - loss: 0.6408 - accuracy: 0.6409

In [ ]:
result_list